In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 8)
!date

%load_ext autoreload
%autoreload 2

Sun Feb 14 09:58:12 PST 2021


# Extract, transform, and "load" data from Facebook Data for Good USA survey

This code requires access to the IHME cluster to run; we have included it for completeness.

In [2]:
def my_etl(df):
    t = pd.DataFrame()

    t['age_group'] = df.D2
    t['male'] = (df.D1 == 1).astype(int)
    
#     t['A1_hh_symptoms'] = df.A1
    t['B2_individual_symptoms'] = df.B2
    t['B2c_new_symptoms'] = df.B2c
    t['B2b_duration_symptoms'] = df.B2b
    
    
    t['test_last_14_days'] = (df.B10 == 1).astype(int)
    t['test_positive'] = (df.B10a == 1).astype(int)
    t['test_negative'] = (df.B10a == 2).astype(int)
    t['test_unknown'] = (df.B10a == 3).astype(int)
    t['test_reason'] = df.B10b
    t['test_required'] = df.B10b.astype(str).str.contains('4')
    
    t['hcw'] = (df.Q64.isin([4,5]))
    t = pd.concat([t, df.filter(regex='^[DQ]')], axis=1)

    t['work_outside_home'] = (df.D10 == 1).astype(int)

    t['activities_past24hrs'] = df.C13
    t['masked_activities_past24hrs'] = df.C13a
    t['mask_past5days'] = df.C14
    
    # race and ethnicity, which is in instrument, but not in data file
    #t['D6'] = df.D6
    #t['D7'] = df.D7
    
    t['weight'] = df.weight
    return t

In [3]:
# !ls -halt /ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/*

In [4]:
# fnames

In [5]:
# test ETL function on the most recent available data file

import glob
fnames = sorted(glob.glob('/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/'
                   'AUTO_EXTRACTED/*/cvid_responses*.csv.gz'), reverse=True)
fname = fnames[0]
print(fname)
df = pd.read_csv(fname, low_memory=False)
%time my_etl(df.iloc[:1_000])

/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2021_02_10_recordedby_2021_02_10.csv.gz
CPU times: user 20 ms, sys: 2.88 ms, total: 22.9 ms
Wall time: 22.9 ms


,age_group,male,B2_individual_symptoms,B2c_new_symptoms,B2b_duration_symptoms,test_last_14_days,test_positive,test_negative,test_unknown,test_reason,...,Q78,Q79,Q80,D10,D11,work_outside_home,activities_past24hrs,masked_activities_past24hrs,mask_past5days,weight
0,6,0,"2,3,6,7,16,18",NaN,NaN,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,0,8,NaN,1,1653.229764
1,4,0,5,NaN,NaN,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,0,"2,4",4.0,1,2540.198738


In [6]:
# parse date out of filenames to be able to record when survey was offered

def my_date(s):
    s1 = s.split('cvid_responses_')[1]
    s2 = s1.split('_recordedby_')[0]
    s3 = s2.replace('_', '-')
    return s3
# results_us['date'] = results_us.index.map(my_date)

In [ ]:
%%time

# loop over all data files from wave 4 of survey
# and extract data from most recent file for each day

data = {}
for fname in fnames:
    data_date = my_date(fname)
    if not data_date in data.keys():
        if data_date <= '2020-09-03':  # earlier waves of survey do not contain key questions for this analysis
            continue
        print(fname)
        df_i = pd.read_csv(fname, low_memory=False)
        %time data[data_date] = my_etl(df_i)

/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2021_02_10_recordedby_2021_02_10.csv.gz
CPU times: user 11 ms, sys: 0 ns, total: 11 ms
Wall time: 11 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2021_02_09_recordedby_2021_02_10.csv.gz
CPU times: user 46.4 ms, sys: 1.16 ms, total: 47.6 ms
Wall time: 47.7 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2021_02_08_recordedby_2021_02_10.csv.gz
CPU times: user 46.9 ms, sys: 14.8 ms, total: 61.7 ms
Wall time: 61.8 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2021_02_07_recordedby_2021_02_10.csv.gz
CPU times: user 53.9 ms, sys: 683 µs, total: 54.6 ms
Wall time: 54.7 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2021_02_06_re

CPU times: user 83.3 ms, sys: 84 µs, total: 83.4 ms
Wall time: 83.6 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2021_01_03_recordedby_2021_01_07.csv.gz
CPU times: user 79.1 ms, sys: 19 µs, total: 79.1 ms
Wall time: 79.3 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2021_01_02_recordedby_2021_01_06.csv.gz
CPU times: user 75.2 ms, sys: 45 µs, total: 75.2 ms
Wall time: 75.4 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2021_01_01_recordedby_2021_01_05.csv.gz
CPU times: user 68.7 ms, sys: 12.9 ms, total: 81.6 ms
Wall time: 81.8 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2020_12_31_recordedby_2021_01_04.csv.gz
CPU times: user 69.7 ms, sys: 0 ns, total: 69.7 ms
Wall time: 69.9 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/CO

CPU times: user 47.8 ms, sys: 3.96 ms, total: 51.7 ms
Wall time: 51.9 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2020_11_26_recordedby_2020_12_18.csv.gz
CPU times: user 47.4 ms, sys: 107 µs, total: 47.5 ms
Wall time: 47.6 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2020_11_25_recordedby_2020_12_18.csv.gz
CPU times: user 49.5 ms, sys: 4.12 ms, total: 53.6 ms
Wall time: 53.7 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2020_11_24_recordedby_2020_12_18.csv.gz
CPU times: user 51.7 ms, sys: 935 µs, total: 52.6 ms
Wall time: 52.8 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2020_11_23_recordedby_2020_12_18.csv.gz
CPU times: user 48.4 ms, sys: 0 ns, total: 48.4 ms
Wall time: 48.5 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDER

CPU times: user 46 ms, sys: 3.01 ms, total: 49 ms
Wall time: 49.2 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2020_10_19_recordedby_2020_12_09.csv.gz
CPU times: user 46 ms, sys: 1.02 ms, total: 47 ms
Wall time: 47.2 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2020_10_18_recordedby_2020_12_09.csv.gz
CPU times: user 45.1 ms, sys: 0 ns, total: 45.1 ms
Wall time: 45.2 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2020_10_17_recordedby_2020_12_09.csv.gz
CPU times: user 44.3 ms, sys: 75 µs, total: 44.4 ms
Wall time: 44.5 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/AUTO_EXTRACTED/latest/cvid_responses_2020_10_16_recordedby_2020_12_09.csv.gz
CPU times: user 41.7 ms, sys: 0 ns, total: 41.7 ms
Wall time: 41.8 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/S

In [ ]:
for k in data.keys():
    data[k]['date'] = k

In [ ]:
df_all = pd.concat(data.values())
df_all

In [10]:
df_all.to_csv('data/fb_data_usa.csv.bz2')

In [12]:
df_all.shape

(7001656, 48)